In [32]:
%autoreload
from lenstools import Observer, Lenses, Sources, nrm
import lenstools as lt

pi = np.pi

## Does the `observe()` function give us the correct angles?

- make 100 sources at random positions in the sky
- calculate their angular positions
- Use angles to calculate their positions
- compare to original positions

In [7]:
N = 100

obs = Observer()
sources = Sources(x = np.random.uniform(-2, 2, (N, 3)))

r = nrm(sources.x)
theta, phi = np.transpose(obs.observe(sources))

x = r[:, np.newaxis] * np.transpose([
    np.sin(theta) * np.cos(phi), 
    np.sin(theta) * np.sin(phi),
    np.cos(theta)
])
print(sum(nrm(x - sources.x) > 1e-14), 'above numerical precision error')

0 above numerical precision error


## Does the lensing formula make sense?

- Make a stationary point-lens 1 kpc away (along y-axis) with mass $10^7$ solar masses
- Make a stationary source 1 kpc beyond that (along y-axis), with an impact parameter of 1 pc (along x-axis)

Unlensed, it should appear a little the midplane, so $\theta = \frac{\pi}{2} - \epsilon$ and $\phi = \frac{\pi}{2}$.

Lensed, it should appear a little more above the midplane, so $\theta = \frac{\pi}{2} - \epsilon - \epsilon'$ and $\phi = \frac{\pi}{2}$.

In [123]:
%autoreload

Ds = .2
Dls = .1
Dl = Ds - Dls
b=1e-6

obs = Observer()
lens = lt.Lens([0,Dl,0], [0,0,0], 'point', 1e7)
source = lt.Source([0,Ds,b], [0,0,0])

unlensed = obs.observe(source)
lensed = obs.observe(source, lens)

print('unlensed theta - pi/2:', unlensed[0]-pi/2)
print('unlensed phi - pi/2:', unlensed[1]-pi/2)
print('lensed theta - pi/2:', lensed[0]-pi/2)
print('lensed phi - pi/2:', lensed[1]-pi/2)
print('lensed - unlensed theta', lensed[0] - unlensed[0])

unlensed theta - pi/2: -5.000000000032756e-06
unlensed phi - pi/2: 0.0
lensed theta - pi/2: -0.00048354169258879587
lensed phi - pi/2: 0.0
lensed - unlensed theta -0.0004785416925887631


Check to see that the lens equation is satisfied:
$$
D_s\tan(\theta) = D_s\tan(\beta) + D_{ls}\tan(\alpha)
$$

In [125]:
Ds_tan_beta = b
theta = pi/2 - lensed[0]
alpha_hat = (1-Dl/Ds) * 4*lt.G_N*lens.M/lt.c**2/b
res = Ds * np.tan(theta) - (Ds_tan_beta + Dls * np.tan(alpha_hat))
print('lens equation is satisfied up to', res)

lens equation is satisfied up to -4.0657581468206416e-20
